In [1]:
import os
os.environ['TOKENIZERS_PARALLELISM'] = 'true'

import pandas as pd
pd.options.display.max_columns = None
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import mlflow

import tensorflow as tf
from tensorflow import keras
import tensorflow_addons as tfa

2023-04-23 14:38:10.915199: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-23 14:38:21.574541: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2023-04-23 14:38:21.574773: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2023-04-23 14:38:21.574788: W tensorfl

In [2]:
from transformers import TFBertForSequenceClassification, BertTokenizerFast

# https://huggingface.co/klue/bert-base

CHECKPOINT = "klue/bert-base" 
bert_base = TFBertForSequenceClassification.from_pretrained(CHECKPOINT, from_pt=True)
tokenizer = BertTokenizerFast.from_pretrained(CHECKPOINT)

2023-04-23 14:38:42.117819: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:967] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-04-23 14:38:42.391526: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:967] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-04-23 14:38:42.391854: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:967] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-04-23 14:38:42.410888: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, 

In [15]:
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

df = pd.read_csv("cleaned_data.csv")
LABEL = 'label'

# df > X, y
X, y = df.drop(LABEL, axis=1), df.loc[:, LABEL]

# encoding label
label_encoder = LabelEncoder()
label_encoder.fit(y)
y = label_encoder.transform(y) # range(7) > ['anger', 'disgust', 'fear', 'happiness', 'neutralism', 'sadness', 'surprise']
n_classes = len(label_encoder.classes_)

# split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, shuffle=True, stratify=y)

In [16]:
train_encoded = tokenizer(list(X_train['text'].values), truncation=True, padding=True, max_length=512, return_tensors='tf')
text_encoded = tokenizer(list(X_test['text'].values), truncation=True, padding=True, max_length=512, return_tensors='tf')

2023-04-23 14:45:58.286557: W tensorflow/tsl/framework/cpu_allocator_impl.cc:82] Allocation of 20378736 exceeds 10% of free system memory.
2023-04-23 14:45:58.345071: W tensorflow/tsl/framework/cpu_allocator_impl.cc:82] Allocation of 20378736 exceeds 10% of free system memory.


In [20]:
from keras.models import Model
from keras.layers import Input

input_ids = Input(batch_shape=(None, 512), dtype=tf.int32, name='input_ids')
input_masks = Input(batch_shape=(None, 512), dtype=tf.int32, name='attention_masks')
embedding = bert_base([input_ids, input_masks])[0]
y_output = keras.layers.Dense(n_classes, activation='softmax')(embedding)

model = Model(inputs=[input_ids, input_masks], outputs=y_output, name='Bert_classification_1')
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['acc'])


In [22]:
model.fit(train_encoded)

ValueError: Cannot generate a hashable key for IteratorSpec(({'input_ids': TensorSpec(shape=(None, 189), dtype=tf.int32, name=None), 'token_type_ids': TensorSpec(shape=(None, 189), dtype=tf.int32, name=None), 'attention_mask': TensorSpec(shape=(None, 189), dtype=tf.int32, name=None)},),) because the _serialize() method returned an unsupproted value of type <class 'transformers.tokenization_utils_base.BatchEncoding'>

In [5]:
train_dataset = tf.data.Dataset.from_tensor_slices((dict(train_encoded), y_train))
test_dataset = tf.data.Dataset.from_tensor_slices((dict(text_encoded), y_test))

2023-04-23 14:39:15.739239: W tensorflow/tsl/framework/cpu_allocator_impl.cc:82] Allocation of 20378736 exceeds 10% of free system memory.
2023-04-23 14:39:24.578557: W tensorflow/tsl/framework/cpu_allocator_impl.cc:82] Allocation of 20378736 exceeds 10% of free system memory.
2023-04-23 14:39:33.387732: W tensorflow/tsl/framework/cpu_allocator_impl.cc:82] Allocation of 20378736 exceeds 10% of free system memory.


In [43]:
from keras.callbacks import EarlyStopping, ModelCheckpoint

def train_model(model, train_params, train_dataset, test_dataset):
    if train_params['optimizer'] == 'Adam':
        optimizer = keras.optimizers.Adam(
            learning_rate=train_params['learning_rate'],
            weight_decay=train_params['weight_decay']
        )
    elif train_params['optimizer'] == 'AdamW':
        optimizer = tfa.optimizers.AdamW(
            learning_rate=train_params['learning_rate'],
            weight_decay=train_params['weight_decay']
        )
    
    model.compile(
        optimizer=optimizer,
        loss=model.compute_loss,
        metrics=['accuracy']
    )
    
    early = EarlyStopping(
        monitor='val_loss',
        min_delta=0,
        patience=train_params['patience'],
        verbose=1,
        restore_best_weights=True
    )
    
    check = ModelCheckpoint(
        filepath='./klue_bert_tuned.h5',
        monitor='val_loss',
        verbose=1,
        save_best_only=True
    )
    
    hist = model.fit(train_dataset,
                     validation_data=test_dataset,
                     epochs=train_params['epochs'],
                     callbacks=[early, check])
    return hist, model
    

In [71]:
CHECKPOINT = "klue/bert-base" 
bert = TFBertForSequenceClassification.from_pretrained(CHECKPOINT,
                                                        from_pt=True,
                                                        num_labels=n_classes)

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertForSequenceClassification: ['bert.embeddings.position_ids']
- This IS expected if you are initializing TFBertForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [74]:
from keras.utils import plot_model

plot_model(bert)

You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) for plot_model to work.


In [44]:
mlflow.set_tracking_uri('./mlruns')
mlflow.set_experiment("Emotion_classfication")

<Experiment: artifact_location='/home/wonhong/workspace/Emotion_Detection/DL/dataset/mlruns/812122781326073109', creation_time=1682257502675, experiment_id='812122781326073109', last_update_time=1682257502675, lifecycle_stage='active', name='Emotion_classfication', tags={}>

In [45]:
train_params = dict(
    optimizer = 'Adam', # Adam, AdamW
    learning_rate=5e-5,
    weight_decay = None, # None, 1e-5 etc
    patience = 5,
    epochs = 30,
)

In [47]:
from sklearn.metrics import accuracy_score

mlflow.tensorflow.autolog()

run_name = "baseline"

with mlflow.start_run(run_name=run_name):
    train_model(model,
                train_params,
                train_dataset,
                test_dataset)
    
    y_pred = model.predict(test_dataset)
    acc = accuracy_score(y_test, y_pred.argmax(axis=1))

2023/04/23 13:55:03 WARNING mlflow.tensorflow: Encountered unexpected error while inferring batch size from training dataset: The layer "tf_bert_for_sequence_classification_1" has never been called and thus has no defined input shape. Note that the `input_shape` property is only available for Functional and Sequential models.


Epoch 1/30


ValueError: in user code:

    File "/usr/local/lib/python3.8/dist-packages/keras/engine/training.py", line 1249, in train_function  *
        return step_function(self, iterator)
    File "/usr/local/lib/python3.8/dist-packages/keras/engine/training.py", line 1233, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/usr/local/lib/python3.8/dist-packages/keras/engine/training.py", line 1222, in run_step  **
        outputs = model.train_step(data)
    File "/transformers/src/transformers/modeling_tf_utils.py", line 1567, in train_step
        y_pred = self(x, training=True)
    File "/usr/local/lib/python3.8/dist-packages/keras/utils/traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "/tmp/__autograph_generated_filewiom9sms.py", line 36, in tf__run_call_with_unpacked_inputs
        retval_ = ag__.converted_call(ag__.ld(func), (ag__.ld(self),), dict(**ag__.ld(unpacked_inputs)),